In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # install transformers
!pip install -q datasets loralib sentencepiece  # tokenizer
!pip -q install bitsandbytes accelerate #8-bit 연산을 위해
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.0

LangChain에서 HuggingFace LLM Wrapper 사용

# Loading Alpaca (영상참고했던 대로하면 모델이 너무 커서 작은 alpaca로 진행)

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("fouadbakour/gpt2-small-alpaca-epoch-1")

base_model = AutoModelForCausalLM.from_pretrained(
    "fouadbakour/gpt2-small-alpaca-epoch-1",
    load_in_8bit = True,
    device_map = 'auto',
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
pipe = pipeline(
    "text-generation",
    model = base_model,
    tokenizer = tokenizer,
    max_length = 256,
    temperature = 0.6,
    top_p = 0.95,
    repetition_penalty = 1.2
) # setup LLM with pipeline

local_llm = HuggingFacePipeline(pipeline = pipe)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

Answer:"""

prompt = PromptTemplate(template = template, input_variables = ["instruction"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm = local_llm)
question = "Can you recommend me a fantasy book similar to Harry Potter?"

print(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Yes, I can! The author of "The Wizarding World" and its sequels has written several fantastic books for children ranging from classic novels such as Hogwarts School Of Witchcraft and wizardry to popular fairy tales like Lord Voldemort's Castle in London or A Time To Remember One Day at Christmas time stories about magical adventures with friends who have never been able come back home safely before - all while staying true-to form throughout their lives!" (HarryPotterBooks).com


In [ ]:
question = "What are the top ten books to read about finance?"

print(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 The Top Ten Books To Read About Finance Are John Locke, Thomas Piketty and Warren Buffett by George Orwell (1957). Also available in English as "The Wealth of Nations", or French for short stories such As Invented By Gouverneur Morris ("Troublesome Tales"). Other popular titles include Stephen King's How Long Have We Been Here?, Jack Kerouac's A Tale Of Two Cities; Ernest Hemingway Presents His Last Book Before He Died at 86th Street Station.; Mark Twain on Writing Good Journalism Volumes 1-4 - Part One with William Faulkner ; David Foster Wallace On Public Speaking vol 5 & 6, Jane Austen Stories Volume 2 'Sketches'; Margaret Atwood Beyond Words volume 3 : Poems from Her Collection From 1900 Through 1945"

 (source)


메모리와 함께 Chat을 설정하기

In [ ]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
# memory go back 4 turns
window_memory = ConversationBufferWindowMemory(k=4)

In [ ]:
conversation = ConversationChain(
    llm = local_llm,
    verbose = True,
    memory = window_memory
)

In [ ]:
conversation.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:

In [ ]:
conversation.prompt.template = """The following is a mean conversation between a human and an AI. The AI is talkative which got anger issues and provides lots of specific details from its context. If the AI does not know the answer to a question, it says get lost.

Current conversation:
{history}
Human: {input}
AI:"""

In [ ]:
conversation.predict(input="Hi my name is Junha")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a mean conversation between a human and an AI. The AI is talkative which got anger issues and provides lots of specific details from its context. If the AI does not know the answer to a question, it says get lost.

Current conversation:

Human: Hi my name is Junha
AI:

> Finished chain.


' Hey there! I\'m sorry for having such trouble today but what do you think? What are your thoughts on this topic right now? My goal in life was always something that would help me succeed as much or more than anything else… Well then let\'s go find out if we can make some progress together before tomorrow evening." (He pauses) "I agree with all those who say no!" He laughs again while looking at his watch impatiently nodding towards himself like he has never seen him smile so brightly since childhood. \n\n"Thank you very Much," said Naoto smiling happily when she saw her new friend walking up beside them next day alone feeling happy about their decision making skills they had been given over by humans ever ago. She looked around nervously wondering why everyone seemed surprised how quickly things have changed though even without knowing exactly where our current predicament lies within us yet still wanting answers made clear through repeated conversations across many different levels 

질문을 하지 않으면 이상한 답변을 함.

In [ ]:
conversation.predict(input="Can you recommend me some funny books?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 270, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.




> Entering new  chain...
Prompt after formatting:
The following is a mean conversation between a human and an AI. The AI is talkative which got anger issues and provides lots of specific details from its context. If the AI does not know the answer to a question, it says get lost.

Current conversation:
Human: Hi my name is Junha
AI:  Hey there! I'm sorry for having such trouble today but what do you think? What are your thoughts on this topic right now? My goal in life was always something that would help me succeed as much or more than anything else… Well then let's go find out if we can make some progress together before tomorrow evening." (He pauses) "I agree with all those who say no!" He laughs again while looking at his watch impatiently nodding towards himself like he has never seen him smile so brightly since childhood. 

"Thank you very Much," said Naoto smiling happily when she saw her new friend walking up beside them next day alone feeling happy about their decision makin

' Yeah'

In [ ]:
conversation.predict(input="What are some funny books I can read")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 286, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.




> Entering new  chain...
Prompt after formatting:
The following is a mean conversation between a human and an AI. The AI is talkative which got anger issues and provides lots of specific details from its context. If the AI does not know the answer to a question, it says get lost.

Current conversation:
Human: Hi my name is Junha
AI:  Hey there! I'm sorry for having such trouble today but what do you think? What are your thoughts on this topic right now? My goal in life was always something that would help me succeed as much or more than anything else… Well then let's go find out if we can make some progress together before tomorrow evening." (He pauses) "I agree with all those who say no!" He laughs again while looking at his watch impatiently nodding towards himself like he has never seen him smile so brightly since childhood. 

"Thank you very Much," said Naoto smiling happily when she saw her new friend walking up beside them next day alone feeling happy about their decision makin

' How'

In [ ]:
conversation.predict(input="재밌는 책 추천해줘")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 317, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.




> Entering new  chain...
Prompt after formatting:
The following is a mean conversation between a human and an AI. The AI is talkative which got anger issues and provides lots of specific details from its context. If the AI does not know the answer to a question, it says get lost.

Current conversation:
Human: Hi my name is Junha
AI:  Hey there! I'm sorry for having such trouble today but what do you think? What are your thoughts on this topic right now? My goal in life was always something that would help me succeed as much or more than anything else… Well then let's go find out if we can make some progress together before tomorrow evening." (He pauses) "I agree with all those who say no!" He laughs again while looking at his watch impatiently nodding towards himself like he has never seen him smile so brightly since childhood. 

"Thank you very Much," said Naoto smiling happily when she saw her new friend walking up beside them next day alone feeling happy about their decision makin

' �'

한국어는 처리를 못함

In [ ]:
conversation.predict(input="What is your name?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a mean conversation between a human and an AI. The AI is talkative which got anger issues and provides lots of specific details from its context. If the AI does not know the answer to a question, it says get lost.

Current conversation:
Human: Can you recommend me some funny books?
AI:  Yeah
Human: What are some funny books I can read
AI:  How
Human: 재밌는 책 추천해줘
AI:  �
Human: What is your name?
AI:  �
Human: What is your name?
AI:

> Finished chain.


' 라글황을니 - Gùng-hong'